<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/tts_zonos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [11]:
# https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-feature-list
!nvcc -code-ls
!nvcc -arch-ls

sm_50
sm_52
sm_53
sm_60
sm_61
sm_62
sm_70
sm_72
sm_75
sm_80
sm_86
sm_87
sm_89
sm_90
compute_50
compute_52
compute_53
compute_60
compute_61
compute_62
compute_70
compute_72
compute_75
compute_80
compute_86
compute_87
compute_89
compute_90


In [12]:
!nvidia-smi

Sun Feb 16 09:10:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   72C    P0             35W /   72W |    4523MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:
!nvidia-smi --query-gpu=compute_cap --format=csv,noheader
# sm_89

8.9


In [29]:
!nvidia-smi --query-gpu=index,name,driver_version,compute_cap,memory.total,memory.used,memory.free,utilization.gpu,utilization.memory --format=csv


index, name, driver_version, compute_cap, memory.total [MiB], memory.used [MiB], memory.free [MiB], utilization.gpu [%], utilization.memory [%]
0, NVIDIA L4, 550.54.15, 8.9, 23034 MiB, 7809 MiB, 14883 MiB, 0 %, 0 %


In [25]:
!lspci -vnn | grep -i nvidia

lspci: Unable to load libkmod resources: error -2
00:03.0 3D controller [0302]: NVIDIA Corporation Device [10de:27b8] (rev a1)
	Subsystem: NVIDIA Corporation Device [10de:16ee]
	Kernel driver in use: nvidia


# install

In [ ]:
!apt install -y espeak-ng

In [ ]:
!pip install uv

In [ ]:
!git clone https://github.com/Zyphra/Zonos.git

In [ ]:
!cd Zonos && uv pip install --system -e . && uv pip install --system -e .[compile]

Note: need restart ipynb

# download model

In [5]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [6]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


Token is valid (permission: write).
The token `write` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `write`


In [7]:
!huggingface-cli download Zyphra/Zonos-v0.1-transformer --quiet --local-dir /content/models/Zyphra/Zonos-v0.1-transformer


/content/models/Zyphra/Zonos-v0.1-transformer


In [8]:
!huggingface-cli download Zyphra/Zonos-v0.1-hybrid --quiet --local-dir /content/models/Zyphra/Zonos-v0.1-hybrid


/content/models/Zyphra/Zonos-v0.1-hybrid


# inference

In [1]:
%cd /content/Zonos

/content/Zonos


In [2]:
import torch

def print_model_params(model: torch.nn.Module, extra_info=""):
    # print the number of parameters in the model
    model_million_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(model)
    print(f"{extra_info} {model_million_params} M parameters")

In [3]:
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict

repo_id = "Zyphra/Zonos-v0.1-hybrid"
model = Zonos.from_pretrained(repo_id, device="cuda").eval()
print_model_params(model)

config.json:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.30G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

Zonos(
  (backbone): MambaSSMZonosBackbone(
    (layers): ModuleList(
      (0): Block(
        (norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (in_proj): Linear(in_features=2048, out_features=3072, bias=False)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (norm2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GatedMLP(
          (fc1): Linear(in_features=2048, out_features=16384, bias=False)
          (fc2): Linear(in_features=8192, out_features=2048, bias=False)
        )
      )
      (1-3): 3 x Block(
        (norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mixer): Mamba2(
          (in_proj): Linear(in_features=2048, out_features=8512, bias=False)
          (conv1d): Conv1d(4352, 4352, kernel_size=(4,), stride=(1,), padding=(3,), groups=4352)
          (act): SiLU()
          (norm): RMSNorm()
    

In [13]:
repo_id = "Zyphra/Zonos-v0.1-transformer"
model = Zonos.from_pretrained(repo_id, device="cuda").eval()
print_model_params(model)

Zonos(
  (backbone): TorchZonosBackbone(
    (layers): ModuleList(
      (0-25): 26 x TransformerBlock(
        (norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mixer): Attention(
          (in_proj): Linear(in_features=2048, out_features=3072, bias=False)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (norm2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): FeedForward(
          (fc1): Linear(in_features=2048, out_features=16384, bias=False)
          (fc2): Linear(in_features=8192, out_features=2048, bias=False)
        )
      )
    )
    (norm_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (prefix_conditioner): PrefixConditioner(
    (project): Linear(in_features=2048, out_features=2048, bias=True)
    (conditioners): ModuleList(
      (0): EspeakPhonemeConditioner(
        (project): Identity()
        (phoneme_embedder): Embedding(189, 2048)
      )
      (1): Passthro

In [4]:
from IPython.display import Audio
Audio("assets/exampleaudio.mp3",autoplay=True)

In [5]:
wav, sampling_rate = torchaudio.load("assets/exampleaudio.mp3")
speaker = model.make_speaker_embedding(wav, sampling_rate)

cond_dict = make_cond_dict(text="Hello, world!", speaker=speaker, language="en-us")
conditioning = model.prepare_conditioning(cond_dict)

codes = model.generate(conditioning)

wavs = model.autoencoder.decode(codes).cpu()
torchaudio.save("sample.wav", wavs[0], model.autoencoder.sampling_rate)

ResNet293_SimAM_ASP_base.pt:   0%|          | 0.00/397M [00:00<?, ?B/s]

ResNet293_SimAM_ASP_base_LDA-128.pt:   0%|          | 0.00/265k [00:00<?, ?B/s]

Generating:   3%|▎         | 83/2588 [00:02<01:11, 35.28it/s]


In [6]:
from IPython.display import Audio
Audio("sample.wav",autoplay=True)

In [11]:
!GRADIO_SHARE=1 uv run gradio_interface.py


| The current ZonosBackbone does not support the hybrid architecture, meaning only the transformer model will be available in the model selector.
| This probably means the mamba-ssm library has not been installed.
share:  True
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://f68e40b4528044b784.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Loading Zyphra/Zonos-v0.1-transformer model...
config.json: 100% 2.52k/2.52k [00:00<00:00, 16.7MB/s]
model.safetensors: 100% 3.25G/3.25G [01:16<00:00, 42.2MB/s]
Zyphra/Zonos-v0.1-transformer model loaded successfully!
Conditioners in this model: ['espeak', 'speaker', 'emotion', 'fmax', 'pitch_std', 'speaking_rate', 'language_id']
Generating:   0% 0/2588 [00:00<?, ?it/s]W0216 09:06:00.944000 4905 .venv/lib/python3.12/site-packages/torch/_inductor/uti